In [1]:
## Initialization and imports
import pandas as pd 
import numpy as np
from scipy import sparse

from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF

## Changing directory to top folder (All programs run from top)
import os
os.chdir('/Users/nschumacher/docs/galvanize/smunch-user-food-analysis')

from db.python_db import run_sql_query

from matplotlib import pyplot as plt
%matplotlib inline  
%config InlineBackend.figure_format='retina'

## Set random seed
np.random.seed(seed=14)

In [2]:
## Selecting pre created table with rating info
df = run_sql_query("SELECT * from noah.cust_ratings")
df.sample(3)

,cust_id,meal_id,meal_name,category,restaurant_name,delivery_tm,meal_rating
14991,0030N00002LQqcxQAD,a050N00000zZfz8QAC,Fajita Veggies Quinoa Bowl,freakyfit,Chupenga,2018-05-22 09:00:00+00:00,5.0
25804,0030N00002LQqMKQA1,a050N00000zZg49QAC,Grilled Teriyaki Salmon Bento,livinglight,Hashi Izakaya & Japanese Kitchen,2017-08-31 09:00:00+00:00,4.0
9812,0030N00002iBdCJQA0,a050N00000zbFdZQAU,Rice Noodle Salad with Chicken Breast,livinglight,Dave B.,2019-02-06 11:20:00+00:00,4.5


In [3]:
df = df[['cust_id', 'meal_name', 'meal_rating']]
df.sample(3)

,cust_id,meal_name,meal_rating
38716,0030N00002LQpCqQAL,Bibimbap with Minced Beef,4.0
37601,0030N00002LQpGrQAL,Fajita Veggies Quinoa Bowl,5.0
58627,0030N00002LQpm9QAD,Lamb and Veggie Curry,3.0


In [4]:
## Pivoting the table to get in format for ALS
table = pd.pivot_table(df,
                       values='meal_rating',
                       index=['cust_id'],
                       columns=['meal_name'],
                       fill_value=0)
table.shape

(4931, 838)

In [5]:
table_empty = table.copy()

In [6]:
## Querry to get the avg meal rating for each meal
querry = '''
SELECT
    product_name as meal_name,
    AVG(rating_score) as avg_meal_rating,
    COUNT(rating_score) as rating_count
FROM 
    bi.executed_order_employee
WHERE
    order_type = 'single' and rating_score IS NOT NULL
GROUP BY
    product_name'''
avg = run_sql_query(querry)
avg.sample(4)

,meal_name,avg_meal_rating,rating_count
636,'Caeser's Delight' Chicken Salad with Anchovie...,3.846154,65
193,Paneer Tikka 'Naanwich' Roll,4.250000,40
765,Club Chicken Bowl with Ceasar Dressing (only t...,5.000000,1
460,Chicken Avocado Arepa,3.818898,381


In [19]:
## Function to assign missing values to avg
def con_avg(x, avg):
    if x == 0:
        return avg
    return x
    
## For each column get avg value and assign it to missing info
for meal in table.columns:
    avg_val = avg.loc[avg['meal_name'] == meal, :].avg_meal_rating.values[0]
    table[meal] = table[meal].apply(con_avg, args=(avg_val,))
    
table.head(2)

meal_name,'Amatriciana' Casarecce Pasta with Guanciale Bacon (Pork cheek),'Be Veggie' Tofu Burger in Brioche Bun,'Berliner Bowl' with Beef-Meatballs,'Caeser's Delight' Chicken Salad with Anchovies & Fresh Veggies in Herb Dressing,'Com Chay' Lemongrass Tofu with Pan-fried Sesame-Veggies,'Com Ga' Lemongrass Chicken with Pan-fried Sesame-Veggies,'New Classic' Beef Burger with Bacon in Brioche Bun,'Pad Lao' Rice Noodles with Chicken,'Pad Lao' Rice Noodles with Tofu,'Pasta Bella Bologna' Fusili in Bolognese Beef Sauce (750ml),...,Wild Rice Bowl with Salmon,Winter Bowl with Yoghurt Dressing (only the bread contains gluten),Wok Chicken in Cocos-Curry,Woked Vegetables & Udon Noodles,Wrap with Mixed Salad & Balsamic Dressing,Za'tar Turkey with Couscous Salad,Zucchini-Aubergine-Paprika Sandwich with Bulgur-Chickpea Salad,Zula Special Chicken Salad,​Beef Kofte & Vermicelli-Rice ​,​​Japanese Bowl with Salmon Ceviche
cust_id,,,,,,,,,,,,,,,,,,,,,
0030N00002LQoyvQAD,4.1,2.769231,4.192308,3.846154,3.2,3.333333,3.429293,3.818182,3.714286,3.648352,...,5.0,2.0,4.125,3.518519,3.190476,2.833333,3.524648,3.181818,3.364103,4.578947
0030N00002LQp3rQAD,4.1,2.769231,4.192308,3.846154,3.2,3.333333,3.000000,3.818182,3.714286,3.648352,...,5.0,2.0,4.125,3.518519,3.190476,2.833333,3.524648,3.181818,3.364103,4.578947


In [8]:
## Taking subset of data to perform NMF on
X = np.round(table.values, 3)
X

array([[4.1  , 2.769, 4.192, ..., 3.182, 3.364, 4.579],
       [4.1  , 2.769, 4.192, ..., 3.182, 3.364, 4.579],
       [4.1  , 2.769, 4.192, ..., 3.182, 3.364, 4.579],
       ...,
       [4.1  , 2.769, 4.192, ..., 3.182, 3.364, 4.579],
       [4.1  , 2.769, 4.192, ..., 3.182, 3.364, 4.579],
       [4.1  , 2.769, 4.192, ..., 3.182, 3.364, 4.579]])

In [42]:
## Creating NMF object
nmf= NMF(max_iter=100, n_components=5, solver='cd')

W = np.round(nmf.fit_transform(X), 5)  ## W matrix: has n hidden user topics
H = np.round(nmf.components_, 4)          ## H matrix: has n hidden meal topics

In [43]:
W

array([[1.22952, 0.04466, 0.15714, 0.06946, 0.0662 ],
       [1.234  , 0.04192, 0.09262, 0.06758, 0.0692 ],
       [1.23589, 0.01924, 0.1224 , 0.07542, 0.     ],
       ...,
       [1.23531, 0.03969, 0.09398, 0.0664 , 0.07106],
       [1.23468, 0.04122, 0.09044, 0.06444, 0.06801],
       [1.23603, 0.04961, 0.07498, 0.05875, 0.06206]])

In [44]:
H

array([[3.3019, 2.2074, 3.371 , ..., 2.5611, 2.6735, 3.6889],
       [0.    , 0.    , 0.6667, ..., 0.0703, 1.0841, 0.    ],
       [0.2346, 0.2017, 0.    , ..., 0.0528, 0.    , 0.2466],
       [0.    , 0.1936, 0.0478, ..., 0.0603, 0.0794, 0.    ],
       [0.    , 0.1476, 0.    , ..., 0.1125, 0.1964, 0.    ]])

In [45]:
preds = np.round(np.dot(W, H), 3)
preds

array([[4.097, 2.769, 4.178, ..., 3.172, 3.354, 4.574],
       [4.096, 2.766, 4.191, ..., 3.18 , 3.364, 4.575],
       [4.11 , 2.767, 4.183, ..., 3.178, 3.331, 4.589],
       ...,
       [4.101, 2.769, 4.194, ..., 3.184, 3.365, 4.58 ],
       [4.098, 2.766, 4.193, ..., 3.181, 3.364, 4.577],
       [4.099, 2.764, 4.203, ..., 3.184, 3.375, 4.578]])

In [46]:
## Convert preds back into user, meal dataframe.
cols, inds = table.columns, table.index
pred_df = pd.DataFrame(preds, index=inds, columns=cols)

### Exloring hidden topics for food.

In [51]:
def hidden_ts(H, cols, show=10):
    num_ts = H.shape[0]
    
    d = {}
    for i in range(num_ts):
        
        ## Sorted indexes of topics by importance
        t_inds = np.argsort(H[i,:])[::-1]
        name = "Topic " + str(i)
        d[name] = cols[t_inds[:show]]
    
    return pd.DataFrame(d)
    
hidden_ts(H, cols, 10)

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4
0,Grilled Aubergine with Feta and Couscous,Bibimbap with Minced Beef,Bibimbap with Minced Beef,Vegan Quinoa Salad with Falafel,Grilled Chicken 'Schaschlik' on Vegetable Rice
1,Mini Goat Cheese & Avocado Burger with Baked ...,Beef Kofte - Coriander Potato - Salad Wrap,Bibimbap with Marinated Tofu,Fajita Veggies Quinoa Bowl,Falafel-Halloumi-Zucchini Patty Vegetarian Pla...
2,Risotto with Chicken,Chicken Breast Salad with Crispy Fried Onions,"Korean 'Bibimbab' with Honey BBQ Beef,",Falafel-Halloumi-Zucchini Patty Vegetarian Pla...,Salmon Steak and Spinach
3,Quinoa Salad with Avocado & Mushrooms,Curried Salmon with Rice,Falafel-Halloumi-Zucchini Patty Vegetarian Pla...,Veggie Quesadilla with Fried Black Beans & Cheese,Roasted Chicken Breast and Bulgur 'Dajaj Steak'
4,Green Tofu Bowl,Grilled Chicken 'Schaschlik' on Vegetable Rice,Warm 'Japchae' Glass noodle Salad and Marinate...,Bibimbap with Marinated Tofu,Chicken Schawarma Platter
5,Power Vegan Bowl with Italian Dressing (only t...,Beef Kebab Platter,Grilled Teriyaki Chicken Bento,Vegan Falafel - Salad Wrap,Lentil Lime-Rice Bowl with Chicken
6,Beef Stew with Bulgur,Spanish Rice Bowl with Pulled Beef,Chicken Avocado Arepa,Coriander Halloumi Salad Platter,Minced Beef 'Moussaka'
7,Chicken Salad,Chicken Schawarma Platter,Rice Plate with Beef Kofte in Tomato Sauce,Vegan Swabian Spinach Ravioli,Coriander Chicken Salad Platter
8,Paella with Marinated Chicken,Chicken Burrito,Pastrami Beef Banh-mi Baguette Sandwich,"Vegetarian Platter with Falafel, Halloumi and ...",Bratwurst on Latvian Sauerkraut
9,Vegan Dal Shabsie,Vegan Quinoa Salad with Falafel,Salmon Poke Bowl in Teriyaki Sauce,Vegan Quinoa Salad with Dijon Mustard Dressing,Chicken Avocado Arepa
